In [1]:
from autodm.llm import get_llm
from autodm.roll import DiceType, Dice
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from tenacity import retry, stop_after_attempt
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core import PromptTemplate

from rich import print

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
llm = get_llm()

In [3]:
DEFAULT_STORYLINE = ""

In [4]:
def parse_rarity(rarity: int):
    if rarity <= 8:
        return "Common, every day item. Nothing fancy. Often do not have any special properties. Should have a name like 'short sword', 'bow', or 'basic wand'."
    elif rarity <= 15:
        return "Uncommon. A bit more uncommon, but still not too hard to find. May have one special property, but nothing fancy. May have a name like 'flaming sword', 'bow of accuracy', or 'wand of light'."
    elif rarity <= 19:
        return "Rare. Not many people have seen one of these. Often have 2-3 special properties. May have a cooler name such as 'whispering blade', 'bow of the hunt', or 'wand of the archmage'."
    else:
        return "Legendary. Extremely rare. Often have 4-5 special properties. Has an awesome name like 'blade of the dragon', 'bow of the storm', or 'wand of the gods'."

In [5]:
class Item(BaseModel):
    "A class to represent an item for D&D"
    name: str = Field(..., title="Name of the item", description="Name of the item")
    description: str = Field(..., title="Description of the item", description="Description of the item")
    quantity: Optional[int] = Field(1, title="Quantity of the item", description="Quantity of the item", ge=0)
    location: Optional[str] = Field(None, title="Location of the item", description="Location of the item. For example, 'worn', 'pack', etc.")
    bonus: Optional[Dict[str, int]] = Field({}, title="Bonus of the item", description="Bonus of the item. For example, {'strengh': 1}")

    @classmethod
    @retry(stop=stop_after_attempt(3))
    def generate(cls, storyline=DEFAULT_STORYLINE, rarity=None, name=None, description=None, quantity=None, location=None, bonus=None):
        if rarity is None:
            rarity = Dice().roll()
            rarity = parse_rarity(rarity)
        property_str = "rarity: {rarity}\n"
        for property, value in zip(['name', 'description', 'quantity', 'location'], [name, description, quantity, location]):
            if value:
                if isinstance(value, dict):
                    value = ', '.join([f"{k}: {v}" for k, v in value.items()])
                property_str += f"{property}: {value}\n"
        if property_str == "":
            property_str = "No properties given. Make up appropriate ones."
        pt = PromptTemplate("""\
Please help create a new D&D item. The item should have the following properties:
{property_str}

Please make sure the item is in line with the following text:
{storyline}
Answer: \
""").partial_format(property_str=property_str)
        program = LLMTextCompletionProgram.from_defaults(llm=get_llm(temperature=1.2), output_cls=Item, prompt=pt)
        return program(storyline=storyline)


In [6]:
class Weapon(Item):
    "A class to represent a weapon for D&D"
    damage_type: str = Field(..., title="Damage type of the weapon", description="Damage type of the weapon")
    damage_roll: DiceType = Field(DiceType.D6, title="Damage roll of the weapon", description="Damage roll of the weapon")
    properties: List[str] = Field([], title="Properties of the weapon", description="Properties of the weapon")
    range_ft: int = Field(5, title="Range of the weapon", description="Range of the weapon", ge=0, le=10)

    @classmethod
    @retry(stop=stop_after_attempt(3))
    def generate(cls, storyline=DEFAULT_STORYLINE, rarity=None, name=None, description=None, quantity=None, location=None, bonus=None):
        if rarity is None:
            rarity_num = Dice().roll()
            rarity = parse_rarity(rarity_num)
        print(rarity)
        property_str = "rarity: {rarity}\n"
        for property, value in zip(['name', 'description', 'quantity', 'location'], [name, description, quantity, location]):
            if value:
                if isinstance(value, dict):
                    value = ', '.join([f"{k}: {v}" for k, v in value.items()])
                property_str += f"{property}: {value}\n"
        if property_str == "":
            property_str = "No properties given. Make up appropriate ones."
        pt = PromptTemplate("""\
Please help create a new D&D melee weapon. The item should have the following properties:
{property_str}

Please make sure the item is in line with the following text:
{storyline}
Answer: \
""").partial_format(property_str=property_str)
        program = LLMTextCompletionProgram.from_defaults(llm=get_llm(), output_cls=Weapon, prompt=pt)
        return program(storyline=storyline)


In [7]:
class RangeWeapon(Weapon):
    "A class to represent a ranged weapon for D&D"
    ammunition: str = Field(None, title="Ammunition of the weapon", description="Ammunition of the weapon. For example 'arrows', 'bolts', etc.")
    damage_roll: DiceType = Field(DiceType.D4, title="Damage roll of the weapon", description="Damage roll of the weapon")
    range_ft: int = Field(5, title="Range of the weapon", description="Range of the weapon", ge=30, le=300)

    @classmethod
    @retry(stop=stop_after_attempt(9))
    def generate(cls, storyline=DEFAULT_STORYLINE, rarity=None, name=None, description=None, quantity=None, location=None, bonus=None):
        if rarity is None:
            rarity_num = Dice().roll()
            rarity = parse_rarity(rarity_num)
        print(rarity)
        property_str = "rarity: {rarity}\n"
        for property, value in zip(['name', 'description', 'quantity', 'location'], [name, description, quantity, location]):
            if value:
                if isinstance(value, dict):
                    value = ', '.join([f"{k}: {v}" for k, v in value.items()])
                property_str += f"{property}: {value}\n"
        if property_str == "":
            property_str = "No properties given. Make up appropriate ones."
        pt = PromptTemplate("""\
Create a new D&D ranged weapon such as a bow, crossbow, javelin, spear, etc. The item should have the following properties:
{property_str}

Please make sure the item is in line with the following text:
{storyline}
Answer: \
""").partial_format(property_str=property_str)
        program = LLMTextCompletionProgram.from_defaults(llm=get_llm(), output_cls=RangeWeapon, prompt=pt)
        return program(storyline=storyline)


In [11]:
print(RangeWeapon.generate())

Uncommon. A bit more uncommon, but still not too hard to find. May have one special property, but nothing fancy. 
May have a name like 'flaming sword', 'bow of accuracy', or 'wand of light'.

RangeWeapon(
    name="Thundercaller's Javelin",
    description='This javelin has been infused with magical energies that call forth thunder when thrown, stunning 
enemies within a radius.',
    quantity=1,
    location=None,
    bonus={'strengh': 2},
    damage_type='Thunder',
    damage_roll=<DiceType.D8: 8>,
    properties=['Stuns enemies within a 10-foot radius upon impact'],
    range_ft=60,
    ammunition='None'
)

In [ ]:
class Spell(BaseModel):
    "A class to represent a spell for D&D"
    name: str = Field(..., title="Name of the spell", description="Name of the spell")
    description: str = Field(..., title="Description of the spell", description="Description of the spell")
    level: int = Field(..., title="Level of the spell", description="Level of the spell", ge=0, le=9)
    damage_roll: DiceType = Field(DiceType.D6, title="Damage roll of the spell", description="Damage roll of the spell")
    num_dice: int = Field(1, title="Number of dice for the spell", description="Number of dice for the spell", ge=1)
    range_ft: int = Field(20, title="Range of the spell", description="Range of the spell", ge=0, le=300)
    components: Optional[List[str]] = Field([], title="Components of the spell", description="Components of the spell")
    duration: str = Field(..., title="Duration of the spell", description="Duration of the spell")

In [ ]:
class Wand(Item):
    "A class to represent a wand for D&D"
    charges: Optional[int] = Field(1, title="Charges of the wand", description="Charges of the wand", ge=0)
    spell: Optional[Spell] = Field(..., title="Spell of the wand", description="Spell of the wand")